# Linear Programming - Factory Production Planning

Problem Description (Summary)

A company manufactures 100 different products across 3 factory locations. The goal is to maximize total profit while respecting resource constraints (materials, labor, machine hours), market demand limits, and setup costs (factories need to be set up for each product).

In [3]:
import pulp
import random

# Seed for reproducibility
random.seed(42)

# Create the LP Maximization problem
model = pulp.LpProblem("Large_Factory_Production_Optimization", pulp.LpMaximize)

# Parameters
num_products = 100
num_factories = 3
resources = ['wood', 'steel', 'plastic', 'labor', 'machine']

# Generate product names
products = [f'Product_{i+1}' for i in range(num_products)]

# Generate factories names
factories = [f'Factory_{i+1}' for i in range(num_factories)]

# Random profit per product (between $20 and $500)
profit = {p: random.randint(20, 500) for p in products}

# Random max demand per product (between 50 and 500 units)
max_demand = {p: random.randint(50, 500) for p in products}

# Resource availability per factory (random large numbers)
resource_avail = {
    f: {
        'wood': random.randint(5000, 15000),
        'steel': random.randint(3000, 12000),
        'plastic': random.randint(2000, 10000),
        'labor': random.randint(7000, 15000),
        'machine': random.randint(6000, 13000)
    }
    for f in factories
}

# Resource consumption per product per factory per unit produced
# Random integers to simulate variability between factories and products
resource_consumption = {
    f: {
        p: {
            r: random.uniform(0.1, 10.0) for r in resources
        }
        for p in products
    }
    for f in factories
}

# Setup cost per product per factory (fixed cost incurred if produce > 0)
setup_cost = {
    f: {p: random.randint(1000, 10000) for p in products}
    for f in factories
}

# Storage capacity per factory (units)
storage_capacity = {f: random.randint(2000, 6000) for f in factories}

# Decision variables:
# x[f,p] = units of product p produced in factory f
x = {
    f: {p: pulp.LpVariable(f"x_{f}_{p}", lowBound=0, cat='Integer') for p in products}
    for f in factories
}

# y[f,p] = binary variable: 1 if product p is produced at factory f (setup), else 0
y = {
    f: {p: pulp.LpVariable(f"y_{f}_{p}", cat='Binary') for p in products}
    for f in factories
}

# Objective: Maximize total profit minus setup costs
model += (
    pulp.lpSum(
        profit[p] * x[f][p]
        for f in factories for p in products
    )
    - pulp.lpSum(
        setup_cost[f][p] * y[f][p]
        for f in factories for p in products
    ),
    "Total_Profit_minus_SetupCosts"
)

# Constraints

# 1. Resource constraints per factory
for f in factories:
    for r in resources:
        model += (
            pulp.lpSum(resource_consumption[f][p][r] * x[f][p] for p in products)
            <= resource_avail[f][r],
            f"Resource_{r}_Limit_in_{f}"
        )

# 2. Demand constraints: total produced (sum across factories) <= demand limit
for p in products:
    model += (
        pulp.lpSum(x[f][p] for f in factories) <= max_demand[p],
        f"Max_Demand_{p}"
    )

# 3. Setup constraints: link x and y (if x[f][p] > 0 then y[f][p] == 1)
# Use Big-M method, M is large number (max demand per product)
for f in factories:
    for p in products:
        M = max_demand[p]
        model += x[f][p] <= M * y[f][p], f"Setup_constraint_{f}_{p}"

# 4. Storage capacity constraints per factory (sum of all units produced <= capacity)
for f in factories:
    model += (
        pulp.lpSum(x[f][p] for p in products) <= storage_capacity[f],
        f"Storage_Capacity_{f}"
    )

# 5. Labor shift constraints per factory: labor cannot exceed 3 shifts per day * 30 days * 8 hrs/shift = 720 hrs max
max_shift_hours = 3 * 30 * 8  # = 720
for f in factories:
    model += (
        pulp.lpSum(resource_consumption[f][p]['labor'] * x[f][p] for p in products)
        <= max_shift_hours,
        f"Labor_Shifts_{f}"
    )

# 6. Additional constraint: Minimum production for key products (simulate contracts)
key_products = random.sample(products, 10)  # randomly select 10 key products
for p in key_products:
    model += (
        pulp.lpSum(x[f][p] for f in factories) >= 20,  # minimum 20 units must be produced
        f"Min_Production_{p}"
    )

# 7. Environmental constraint: total plastic used <= 25000 units
model += (
    pulp.lpSum(
        resource_consumption[f][p]['plastic'] * x[f][p]
        for f in factories for p in products
    )
    <= 25000,
    "Plastic_Environmental_Limit"
)

# 8. Market share constraint: Product_1's production must be at least 5% of total production units
total_units = pulp.lpSum(x[f][p] for f in factories for p in products)
prod1_units = pulp.lpSum(x[f]['Product_1'] for f in factories)
model += prod1_units >= 0.05 * total_units, "Product_1_Market_Share"

# 9. Binary variable sum constraint: only 60 products can be produced per factory (setup limit)
for f in factories:
    model += pulp.lpSum(y[f][p] for p in products) <= 60, f"Max_Setups_{f}"

# 10. Piecewise production constraint (simulate production stages):
# For Product_10 at Factory_1: production units must be between 50 and 150 or zero (either produce none or produce within that range)
p = 'Product_10'
f = 'Factory_1'
z1 = pulp.LpVariable(f"z1_{f}_{p}", cat='Binary')  # binary for zero production
z2 = pulp.LpVariable(f"z2_{f}_{p}", cat='Binary')  # binary for production range
model += z1 + z2 == 1, f"Production_Choice_{f}_{p}"  # exactly one choice
model += x[f][p] <= 150 * z2, f"Upper_bound_{f}_{p}"
model += x[f][p] >= 50 * z2, f"Lower_bound_{f}_{p}"
model += x[f][p] <= 0 + 1000000 * z1, f"Zero_production_{f}_{p}"

# Solve the model
print("Solving large-scale LP model, please wait ...")
model.solve()

# Print solution status
print("Status:", pulp.LpStatus[model.status])

# Print production plan summary
print("\nProduction Plan Summary:")
total_profit = pulp.value(model.objective)
print(f"Total Profit (after setup costs): ${total_profit:.2f}")

for f in factories:
    print(f"\n{f} production:")
    for p in products:
        units = x[f][p].varValue
        if units > 0:
            print(f" - {p}: {int(units)} units")

print("\nSetup decisions (products produced per factory):")
for f in factories:
    count = sum(y[f][p].varValue for p in products)
    print(f"{f}: {int(count)} products set up")


Solving large-scale LP model, please wait ...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/roniya/.local/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/f0150cf5020c47aa97652d14aa829759-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/f0150cf5020c47aa97652d14aa829759-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 445 COLUMNS
At line 6188 RHS
At line 6629 BOUNDS
At line 7232 ENDATA
Problem MODEL has 440 rows, 602 columns and 3938 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 977812 - 0.00 seconds
Cgl0003I 2 fixed, 0 tightened bounds, 36 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 12 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 8 strengthened rows, 0 substitutions
Cgl0004I processed model has 411 rows, 598 columns (598 integer 

Cbc0010I After 0 nodes, 1 on tree, -913498 best solution, best possible -920882.1 (0.16 seconds)
Cbc0012I Integer solution of -913769 found by rounding after 460 iterations and 17 nodes (0.26 seconds)
Cbc0012I Integer solution of -913848 found by rounding after 464 iterations and 18 nodes (0.26 seconds)
Cbc0038I Full problem 411 rows 598 columns, reduced to 3 rows 4 columns
Cbc0038I Full problem 443 rows 598 columns, reduced to 213 rows 348 columns
Cbc0044I Reduced cost fixing - 213 rows, 348 columns - restarting search
Cbc0012I Integer solution of -913848 found by Previous solution after 0 iterations and 0 nodes (0.29 seconds)
Cbc0038I Full problem 213 rows 348 columns, reduced to 17 rows 19 columns
Cbc0031I 25 added rows had average density of 59.52
Cbc0013I At root node, 25 cuts changed objective from -962936.04 to -919698.04 in 9 passes
Cbc0014I Cut generator 0 (Probing) - 39 row cuts average 2.2 elements, 41 column cuts (41 active)  in 0.004 seconds - new frequency is 1
Cbc0014I C